# Project 3, Part 1
Sarah Musselwhite
10/12/2022

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

os.makedirs('Data/',exist_ok=True) 
os.listdir("Data/")

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [2]:
basics_url= 'https://datasets.imdbws.com/title.basics.tsv.gz'
aka_url= 'https://datasets.imdbws.com/title.akas.tsv.gz'
rate_url= 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
aka = pd.read_csv(aka_url, sep='\t', low_memory=False)
ratings = pd.read_csv(rate_url, sep='\t', low_memory=False)

In [4]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
aka.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1918
1,tt0000002,5.8,260
2,tt0000003,6.5,1726
3,tt0000004,5.6,173
4,tt0000005,6.2,2541


In [7]:
# Replace '\N' with np.nan
basics.replace({'\\N':np.nan}, inplace=True)
aka.replace({'\\N':np.nan},inplace=True)
basics.replace({'\\N':np.nan},inplace=True)

In [8]:
aka = aka[(aka["region"] == "US")]
aka.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [9]:
keepers_ratings =ratings['tconst'].isin(aka['titleId'])
ratings = ratings[keepers_ratings]
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1918
1,tt0000002,5.8,260
4,tt0000005,6.2,2541
5,tt0000006,5.1,175
6,tt0000007,5.4,797


In [10]:
# Drop any rows that have null values in either genre or runtimeMinutes.
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

In [11]:
# Drop all rows where titleType!=Movie. Keep only titleType==Movie
is_Movie = basics['titleType'] == 'movie'
basics = basics[is_Movie]

In [12]:
# Drop all rows where the start year is not between 2000-2022. Keep startYear 2000-2022
basics.info()

#converting str to int
print("\nData Type before conversion : ", basics['startYear'].dtype)
basics['startYear'] = pd.to_numeric(basics['startYear'])
print("\nData Type after conversion : ", basics['startYear'].dtype)

#Now we can make and apply our filters
filter2000 = basics['startYear'] >= 2000 
filter2022 = basics['startYear'] <=2022
basics = basics[filter2000 & filter2022]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370066 entries, 8 to 9306050
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          370066 non-null  object
 1   titleType       370066 non-null  object
 2   primaryTitle    370066 non-null  object
 3   originalTitle   370066 non-null  object
 4   isAdult         370066 non-null  object
 5   startYear       364140 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  370066 non-null  object
 8   genres          370066 non-null  object
dtypes: object(9)
memory usage: 28.2+ MB

Data Type before conversion :  object

Data Type after conversion :  float64


In [13]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [14]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers_basics = basics['tconst'].isin(aka['titleId'])

basics = basics[keepers_basics]

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86770,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93906,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [15]:
ratings.replace({'\\N':np.nan}, inplace=True)
keepers = ratings['tconst'].isin(aka['titleId'])
ratings = ratings[keepers]
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1918
1,tt0000002,5.8,260
4,tt0000005,6.2,2541
5,tt0000006,5.1,175
6,tt0000007,5.4,797


In [16]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
aka.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)